<a href="https://colab.research.google.com/github/KyoungmiKwon/Bigdata_Training_at_ITwill/blob/main/Team_Project/C05_KM_SentimentDict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 설치

In [3]:
#Konlpy 설치
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [4]:
#Mecab 설치
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [5]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Processing /tmp/mecab-python-0.996
  Created wheel for mecab-python: filename=mecab_python-0.996_ko_0.9.2-cp37-cp37m-linux_x86_64.whl size=141812 sha256=24a0c8cd1b0456a166b87446f9a4764d55004ebd427072eeda64e37e13d09b5e
  Stored in directory: /root/.cache/pip/wheels/99/75/a6/e9e73a1dbd73579383644942ef18a6d17ad728a3052a1147fb
Successfully built mecab-python
  Found existing installation: mecab-python 0.996-ko-0.9.2
    Uninstalling mecab-python-0.996-ko-0.9.2:
      Successfully uninstalled mecab-python-0.996-ko-0.9.2


In [6]:
!git clone https://github.com/entelecheia/eKoNLpy.git
!pip install /content/eKoNLpy

fatal: destination path 'eKoNLpy' already exists and is not an empty directory.
Processing ./eKoNLpy
  Created wheel for eKoNLPy: filename=eKoNLPy-0.5.30-cp37-none-any.whl size=14437140 sha256=3f0e2e0241f58e6f4f4f33e7a0946d88c461952fb38e679ece44d74a5a9524af
  Stored in directory: /tmp/pip-ephem-wheel-cache-lu9kr_b2/wheels/42/07/b6/a197e00efdc4534f4c1d281bd028c47764df1a63a8b311aa3d
Successfully built eKoNLPy
  Found existing installation: eKoNLPy 0.5.30
    Uninstalling eKoNLPy-0.5.30:
      Successfully uninstalled eKoNLPy-0.5.30


# import

In [7]:
from ekonlpy.tag import Mecab
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Basic Sentiment Dictionary 생성
    
    Dictionary class for
    Korean Sentiment Analysis.

In [8]:
import os
from konlpy.tag import Kkma
from ekonlpy.sentiment.base import LEXICON_PATH, BaseDict
from ekonlpy.sentiment.utils import KTokenizer
from ekonlpy.tag import Mecab

class KSAA(BaseDict):
    '''
    Dictionary class for
    Korean Sentiment Analysis.
    '''

    def init_tokenizer(self, kind=None):
        self._tokenizer = KTokenizer(self._poldict)

    def init_dict(self, kind=None, intensity_cutoff=None):
        path = os.path.join('/content/drive/MyDrive/Colab Notebooks/Project/polar_dictionary.csv')
        with open(path, encoding='utf-8') as f:
            for line in f:
                word = line.split(',')
                w = word[0]
                if w == 'ngram':
                    continue
                
                p = float(word[6].strip())
                n = float(word[3].strip())
                s = p - n
                if len(w) > 1:
                    if s > 0:
                        self._posdict[w] = 1
                        self._poldict[w] = s
                    elif s < 0:
                        self._negdict[w] = -1
                        self._poldict[w] = s


class KOSAC(object):
    def __init__(self):
        self._loaddic()
        self._tagger = Mecab()
        self._ngram = 3
        self._delimiter = ';'
        self._skiptags = ['SF', 'SP', 'SS', 'SE', 'SO', 'SW', 'UN', 'UV', 'UE', 'OL', 'OH', 'ON']

    def _loaddic(self):
        self._polarity = self._loadfile('/content/drive/MyDrive/Colab Notebooks/Project/polar_dictionary.csv')
        self._expressive = self._loadfile(os.path.join(LEXICON_PATH, 'kosac', 'expressive-type.csv'))
        self._intensity = self._loadfile(os.path.join(LEXICON_PATH, 'kosac', 'intensity.csv'))

    def _loadfile(self, file_path, delimiter=','):
        vocab = {}
        if os.path.isfile(file_path):
            with open(file_path) as f:
                for lno, line in enumerate(f):
                    # skip header
                    if lno == 0:
                        headers = line.strip().split(delimiter)
                    else:
                        if len(line) > 0:
                            row = line.strip().split(delimiter)
                            ngram = row[0]
                            # ngram_split = tuple(ngram.split(';'))
                            data = {}
                            for i, header in enumerate(headers):
                                if i > 0:
                                    data[header] = row[i]
                            vocab[ngram] = data
        return vocab

    def morpheme(self, dataset):
        return self.align_morpheme(self._tagger.pos(dataset))

    def align_morpheme(self, morpheme):
        return ['{}/{}'.format(w, t) for w, t in morpheme]

    def percentage(self, obj):
        return {k: v / sum(obj.values()) for k, v in obj.items()}

    def calc(self, keypairs, source, target, func):
        for keypair in keypairs:
            sourcekey = keypair[0]
            targetkey = keypair[1]
            if sourcekey in source:
                sourcedata = source[sourcekey]
                sourcedata = float(sourcedata)
                target[targetkey] = func(sourcedata, target[targetkey])
        return target

    def match(self, data, pairdata, keypairs):
        ret = {k[1]: 0 for k in keypairs}
        for m in data:
            if m in pairdata:
                currentdata = pairdata[m]
                ret = self.calc(keypairs, currentdata, ret,
                                lambda s, t: t + s)
        return self.percentage(ret)

    def polarity(self, data):
        return self.match(data,
                          self._polarity,
                          [['COMP', 'com'],
                           ['POS', 'pos'],
                           ['NEG', 'neg'],
                           ['NEUT', 'neut'],
                           ['None', 'none']])

    def intensity(self, data):
        return self.match(data,
                          self._intensity,
                          [['High', 'high'],
                           ['Low', 'low'],
                           ['Medium', 'medium'],
                           ['None', 'none']])

    def expressive(self, data):
        return self.match(data,
                          self._expressive,
                          [['dir-action', 'dir-action'],
                           ['dir-explicit', 'dir-explicit'],
                           ['dir-speech', 'dir-speech'],
                           ['indirect', 'indirect'],
                           ['writing-device', 'writing-device']])

    def analyze(self, dataset):
        dataset = self.parse(dataset)
        ret = {}
        for analysis in ['polarity', 'intensity', 'expressive']:
            func = getattr(self, analysis)
            ret[analysis] = func(dataset)
        return ret

    def parse(self, dataset):
        tokens = []
        if type(dataset) == list:
            for t in dataset:
                tokens += self.morpheme(t)
        elif type(dataset) == str:
            tokens = self.morpheme(dataset)
        else:
            raise ValueError('The dataset has to be string or list of string type.')

        return self.ngramize(tokens)

    def ngramize(self, tokens):
        ngram_tokens = []
        tokens = [w for w in tokens if w.split('/')[1] not in self._skiptags]
        for pos in range(len(tokens)):
            for gram in range(1, self._ngram + 1):
                token = self.get_ngram(tokens, pos, gram)
                if token:
                    ngram_tokens.append(token)
        return ngram_tokens

    def get_ngram(self, tokens, pos, gram):
        if pos < 0:
            return None
        if pos + gram > len(tokens):
            return None
        token = tokens[pos]
        for i in range(1, gram):
            token += self._delimiter + tokens[pos + i]
        return token

## 감성 극성계산 위한 함수들 생성

In [9]:
import abc
from ekonlpy.utils import installpath

LEXICON_PATH = '%s/data/lexicon' % installpath


class BaseDict(object):
 

    __metaclass__ = abc.ABCMeta

    TAG_POL = 'Polarity'
    TAG_SUB = 'Subjectivity'
    TAG_POS = 'Positive'
    TAG_NEG = 'Negative'

    EPSILON = 1e-6

    def __init__(self, tokenizer=None, kind=None, intensity_cutoff=None):
        self._posdict = {}
        self._negdict = {}
        self._poldict = {}
        self._intensity_cutoff = intensity_cutoff
        self.init_dict(kind, intensity_cutoff)
        if tokenizer is None:
            self.init_tokenizer(kind)
        else:
            self._tokenizer = tokenizer

        assert len(self._posdict) > 0 and len(self._negdict) > 0

    def tokenize(self, text):

        return self._tokenizer.tokenize(text)


    @abc.abstractmethod
    def init_tokenizer(self, kind):
        pass

    @abc.abstractmethod
    def init_dict(self, kind, intensity_cutoff):
        pass

    def _get_score(self, term, by_count=True):

        if by_count:
            if term in self._posdict.keys():
                return self._posdict[term]
            elif term in self._negdict.keys():
                return self._negdict[term]
            else:
                return 0
        else:
            if term in self._poldict.keys():
                return self._poldict[term]
            else:
                return 0

    def get_score(self, terms, by_count=True):

        assert isinstance(terms, list) or isinstance(terms, tuple)
        score_li = [self._get_score(t, by_count) for t in terms]
        pos_score_li = [s for s in score_li if s > 0]
        neg_score_li = [s for s in score_li if s < 0]

        s_pos = sum(pos_score_li)
        s_neg = sum(neg_score_li)

        s_pol = (s_pos + s_neg) * 1.0 / (((s_pos - s_neg) if by_count else len(score_li)) + self.EPSILON)
        s_sub = (len(pos_score_li) + len(neg_score_li)) * 1.0 / (len(score_li) + self.EPSILON)

        return {self.TAG_POS: s_pos,
                self.TAG_NEG: s_neg,
                self.TAG_POL: s_pol,
                self.TAG_SUB: s_sub}

# Tokenkizer 생성 using in Sentiment dictionary

    An abstract class for tokenize text.

    The default tokenizer for KSA sub class.
    The output of the tokenizer is tagged by Kkma.   
    

In [10]:
import abc
import re
import nltk
import os
from ekonlpy.tag import Mecab
from ekonlpy.sentiment.base import LEXICON_PATH
from konlpy.tag import Kkma


class BaseTokenizer(object):
    '''
    An abstract class for tokenize text.
    '''

    __metaclass__ = abc.ABCMeta

    @abc.abstractmethod
    def tokenize(self, text):
        '''Return tokenized temrs.
        
        :type text: str
        
        :returns: list 
        '''
        pass

    # @abc.abstractmethod
    # def ngramize(self, tokens):
    #     '''Return n-gramized temrs.
    #
    #     :type tokens: list of tokens
    #
    #     :returns: list
    #     '''
    #     pass


class KTokenizer(BaseTokenizer):
    '''
    The default tokenizer for KSA sub class.
    The output of the tokenizer is tagged by Kkma.
    '''

    def __init__(self, vocab=None):
        self._tagger = Kkma()
        self._vocab = vocab
        self._min_ngram = 1
        self._ngram = 3
        self._delimiter = ';'
        self._skiptags = ['SF', 'SP', 'SS', 'SE', 'SO', 'SW', 'UN', 'UV', 'UE', 'OL', 'OH', 'ON']

    def tokenize(self, text):
        tokens = []
        if type(text) == list:
            for t in text:
                tokens += self.morpheme(t)
        elif type(text) == str:
            tokens = self.morpheme(text)
        else:
            raise ValueError('The dataset has to be string or list of string type.')

        return self.ngramize(tokens)

    def ngramize(self, tokens):
        ngram_tokens = []
        tokens = [w for w in tokens if w.split('/')[1] not in self._skiptags]
        for pos in range(len(tokens)):
            for gram in range(1, self._ngram + 1):
                token = self.get_ngram(tokens, pos, gram)
                if token:
                    if self._vocab is None:
                        ngram_tokens.append(token)
                    else:
                        if token in self._vocab:
                            ngram_tokens.append(token)
        return ngram_tokens

    def get_ngram(self, tokens, pos, gram):
        if pos < 0:
            return None
        if pos + gram > len(tokens):
            return None
        token = tokens[pos]
        for i in range(1, gram):
            token += self._delimiter + tokens[pos + i]
        return token

    def morpheme(self, dataset):
        return self.align_morpheme(self._tagger.pos(dataset))

    def align_morpheme(self, morpheme):
        return ['{}/{}'.format(w, t) for w, t in morpheme]

# 데이터 불러오기

In [11]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Project/news_dt.xlsx'
data = pd.read_excel(file_path,sheet_name='Sheet1',header=0)
data[:2]

,No,Code,Date,Journal,Title,Text,URL,Sample Y/N,In Charge,S,seperate
0,14,5930,2021.04.14 17:40,한국경제,"삼성 '13조 상속세'…연부연납 활용, 올해 낼 2兆 중 1.2兆 대출...",삼성 조 상속세 이재용 부회장 등 재원 마련은보유주식 담보대출 적용나머지는 부동...,https://finance.naver.com/item/news_read.nhn?a...,A,JY,0,"['상속세', '이재용', '재원', '마련', '보유', '적용', '나머지', ..."
1,15,5930,2021.04.14 16:49,이데일리,"삼성, 車반도체 생산 가능성은…""수익성·기존 고객사 문제로 어려워""",인텔 반도체 회의 직후 반도체 생산 생산 전망삼성 뛰어들기 어려울 것수익성 낮고 제...,https://finance.naver.com/item/news_read.nhn?a...,Y,SW,-1,"['회의', '직후', '생산', '생산', '전망', '수익성', '제조', '인..."


In [12]:
data.groupby(['S'])['No'].count()

S
-1    308
 0    371
 1    675
Name: No, dtype: int64

# 셋트 분리

In [13]:
# 부정,중립,긍정으로 할 경우
X = data['Text']
y = data['S']

In [14]:
# 긍정, 긍정이 아닌 경우 : 이진분류로 치환시
y_1 = data['S'].replace(0,-1)
y = pd.DataFrame(y_1)

In [15]:
X_tr, X_ts, y_tr, y_ts = train_test_split(X,y,             
                                          test_size = 0.2,
                                          stratify = y)

In [16]:
std_data = pd.concat([X_tr,y_tr],axis=1, ignore_index=False)
std_data[10:15]

,Text,S
34,중국군에 들어간 생산칩에대량살상무기 안보논리 대응중국 반도체 설계 블랙리스트 등...,-1
605,서 롤러블폰 공개해 주목올 상반기 출시로부터 플렉시블 공급가격적 측면 고려예상 ...,-1
804,삼성전자가 지난해 미중 무역갈등과 신종 코로나바이러스 감염증코로나 등 불확실성 속에...,1
209,삼성 반도체 투자 순위년 위년 위로 올라서 년째 위미래 먹거리 확보 위한 투자...,-1
922,삼성전자는 지난해 연간 영업이익이 조억원으로 전년 대비 증가한 것으로 잠정 집계됐...,1


In [17]:
len(data['Text'])

1354

In [ ]:
ksaa = KSAA()

pola_std_text = []
for i in range(len(std_data['Text'])): # 기사 하나 씩

    polar = 0
    for k in std_data['Text']:         # 하나의 기사에서 한 문장   
        tokens = ksaa.tokenize(k)

        token_lst = []
        for a in tokens:                    #> ['가/VV', '되/XSV', '확산/NNG', '따르/VV', '거리/NNG', '기/NNG', '정착/NNG', '시장/NNG', '이/JKS;불/VV', '불/VV', '등/NNB', '등/NNB;을/JKO'......]
            one_dot = a.split('/')          #> ['확산', 'NNG']
            if len(one_dot[0]) != int(1):   #>  '확산'
                join_dot= '/'.join(one_dot) #> 확산/NNG
                token_lst.append(join_dot)
        
        if len(token_lst) < int(2) :
            token_lst=[]
        
        # print(token_lst)
        score = ksaa.get_score(token_lst)   #> {'Positive': 55, 'Negative': -18, 'Polarity': 0.506849308125352, 'Subjectivity': 0.9999999863013701}
        polar += score['Polarity']
        sen_num = len(k) #> 문장 수
    
    pola_std_text.append(polar/sen_num)

std_data['polarity_std'] = pola_std_text
